In [33]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [34]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [35]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub029_FIF.fif'
    )
)

subID = 'Sub029'
fft_name = str(subID) + '_'
print(fft_name)

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub029_FIF.fif...
Isotrak not found
    Range : 0 ... 80936 =      0.000 ...   323.744 secs
Ready.
Sub029_


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_13220\4047204508.py:3: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub029_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  test_raw = mne.io.read_raw_fif(os.path.join(


In [36]:
reload(dat_preproc)
x = test_raw.get_data() #getting only RSTN channel
x1 = x[1,:]

peakMed = 83
peakStim = 71

dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 

datall = [dat_ngam, dat_subh] 
labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz']
print(labels)

['Peak83Hz', 'Peak71Hz']


In [37]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [38]:
%matplotlib qt
sm_signal_np = np.empty(shape = (2, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [39]:
x2 = x[5, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [40]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1, label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [41]:
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal,
    columns = ['Spontan','StimOn','StimVec'],
    )
print(sm_analSignal_df)

        Spontan    StimOn  StimVec
0      1.443568  0.741575      0.0
1      1.442884  0.739543      0.0
2      1.442797  0.738194      0.0
3      1.442759  0.737005      0.0
4      1.442646  0.735650      0.0
...         ...       ...      ...
80433  1.139136  0.750086      NaN
80434  1.140602  0.751345      NaN
80435  1.142207  0.752562      NaN
80436  1.143346  0.753550      NaN
80437  1.144559  0.754643      NaN

[80438 rows x 3 columns]


In [42]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Crop Analytic Signal to Epochs of Interest

In [43]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub029_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub029'
fft_name = str(subID) + '_'
print(fft_name)

       Unnamed: 0   Spontan    StimOn  StimVec
0               0  1.443568  0.741575      0.0
1               1  1.442884  0.739543      0.0
2               2  1.442797  0.738194      0.0
3               3  1.442759  0.737005      0.0
4               4  1.442646  0.735650      0.0
...           ...       ...       ...      ...
80433       80433  1.139136  0.750086      NaN
80434       80434  1.140602  0.751345      NaN
80435       80435  1.142207  0.752562      NaN
80436       80436  1.143346  0.753550      NaN
80437       80437  1.144559  0.754643      NaN

[80438 rows x 4 columns]
Sub029_


In [44]:
%matplotlib qt

plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'])
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'])
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--')


In [45]:
#CROP IT

med1 = 0
med2 = 10
sub_pt = 35000
time_on = sub_pt - 25*250
time_off = sub_pt + 15*250

sm_anal1 = sm_anal.iloc[med1*250 : med2*250]
sm_anal2 = sm_anal.iloc[time_on : time_off]
sm_toplot = pd.concat([sm_anal1, sm_anal2])
print(sm_toplot)

       Unnamed: 0   Spontan    StimOn  StimVec
0               0  1.443568  0.741575      0.0
1               1  1.442884  0.739543      0.0
2               2  1.442797  0.738194      0.0
3               3  1.442759  0.737005      0.0
4               4  1.442646  0.735650      0.0
...           ...       ...       ...      ...
38745       38745  0.699384  1.040456      4.2
38746       38746  0.698182  1.040389      4.2
38747       38747  0.697103  1.040289      4.2
38748       38748  0.696141  1.040150      4.2
38749       38749  0.695293  1.039967      4.2

[12500 rows x 4 columns]


In [54]:
35*250

8750

In [56]:
# CROP ONLY THE TRANSITION E.G. BETWEEN 0.5 - 1.0mA
this_sig = sm_toplot

fig, ax1 = plt.subplots(figsize = (8,6))
ax2 = ax1.twinx()

ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['Spontan'], label = 'Med-Induced')
ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['StimOn'], label = 'Subharmonic')
ax2.plot(np.arange(0, this_sig.shape[0]), this_sig['StimVec'], ls='--', color = 'grey')
ax1.axvline(x = 8750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
ax1.axvline(x = 2500, color = 'grey', lw=3, alpha = 0.2)


ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xlabel('Time [samples]')
ax1.set_title(str(subID))


Text(0.5, 1.0, 'Sub029')

In [57]:
fft_fig = os.path.join(project_path, 'figures','anal_signal/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalTRANSITION',dpi = 150)
this_sig.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalTRANSITION.csv')

### Average Cropped Analytic Signal and Plot

In [58]:
med_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllMed_analSignalTRANSITION.csv'
))

stim_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllStim_analSignalTRANSITION.csv'
))

print(stim_anal)

         sub005    sub006    sub009    sub014    sub017    sub029
0      0.796159  0.881777  0.851086  0.526401  0.806174  0.741575
1      0.796054  0.879257  0.848726  0.527014  0.804659  0.739543
2      0.796050  0.876840  0.846447  0.527712  0.804158  0.738194
3      0.796008  0.874098  0.844113  0.528500  0.803850  0.737005
4      0.795919  0.871046  0.841790  0.529381  0.803369  0.735650
...         ...       ...       ...       ...       ...       ...
17495  0.509684  1.596356  2.327574  2.518225  1.330336       NaN
17496  0.510192  1.597226  2.321686  2.517199  1.328064       NaN
17497  0.510687  1.598283  2.316014  2.516238  1.325696       NaN
17498  0.511173  1.599522  2.310550  2.515339  1.323238       NaN
17499  0.511650  1.600936  2.305284  2.514498  1.320694       NaN

[17500 rows x 6 columns]


In [59]:
med_anal_mean = np.nanmean(med_anal,1)
med_anal_sem = stats.sem(med_anal,1, nan_policy='omit')

stim_anal_mean = np.nanmean(stim_anal,1)
stim_anal_sem = stats.sem(stim_anal,1, nan_policy='omit')


In [60]:
%matplotlib qt
plt.plot(np.arange(0,17500), med_anal_mean, label = 'Med-Induced FTG')
plt.fill_between(np.arange(0,17500), med_anal_mean-med_anal_sem, med_anal_mean+med_anal_sem, alpha = 0.2)

plt.plot(np.arange(0,17500 ), stim_anal_mean, color = 'red', label = 'Stim-Induced FTG')
plt.fill_between(np.arange(0,17500), stim_anal_mean-stim_anal_sem, stim_anal_mean+stim_anal_sem, color = 'red', alpha = 0.2)


plt.axvline(x = 13750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
plt.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)

plt.legend()
plt.xlabel('Time [samples]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

Text(0, 0.5, 'Z-scored Smoothed Analytic Signal')

In [61]:
fft_fig = os.path.join(project_path, 'results')

plt.savefig(str(fft_fig)+'AVG_AnalCropped',dpi = 200)

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values